<a href="https://colab.research.google.com/github/dtoralg/IE_Calidad_ML/blob/main/Ejercicios/Modulo%205/Modulo_5_Ejercicio_3_Grid_vs_Random_Search_Resuelto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Ejercicio 3: ¿Grid Search o Random Search? Elige tu estrategia de optimización**
**Optimización de hiperparámetros en Random Forest para clasificación multiclase**

### **Introducción**
En este ejercicio aprenderás a aplicar y comparar dos estrategias de búsqueda de hiperparámetros: **Grid Search** y **Random Search**.

Ambas son técnicas comunes para optimizar el rendimiento de un modelo de machine learning, pero con enfoques y costes computacionales distintos.
Utilizaremos un modelo de **Random Forest** aplicado a un problema de clasificación multiclase en un dataset farmacéutico realista.

In [1]:
# Celda 1: Carga de librerías y configuración
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import classification_report, f1_score
from sklearn.impute import SimpleImputer
sns.set(style='whitegrid')

In [2]:
# Celda 2: Cargar el dataset
url = 'https://github.com/dtoralg/IE_Calidad_ML/raw/main/Data/control_calidad_farmaceutico.csv'
df = pd.read_csv(url)
df.head()

,ID_lote,pH_disolución,Viscosidad,Temperatura_proceso,Tiempo_mezcla,Proveedor_excipientes,Tipo_envase,Condiciones_almacenamiento,Fecha_producción,Tipo_defecto
0,1,7.248357,1.236467,86.849390,60.324385,Proveedor_8,Envase_6,Ambiente,2019-12-04,Rotura
1,2,6.930868,1.685664,79.458674,52.219221,Proveedor_4,Envase_2,Temperatura controlada,2022-09-22,Sin defecto
2,3,7.323844,1.161342,74.584565,47.979095,Proveedor_5,Envase_2,Ambiente,2020-01-03,Rotura
3,4,7.761515,1.847235,86.959804,66.975391,Proveedor_1,Envase_6,Ambiente,2023-09-12,Rotura
4,5,6.882923,1.275656,79.434116,63.796312,Proveedor_5,Envase_6,Ambiente,2023-02-06,Sin defecto


In [3]:
# Celda 3: Preprocesamiento del dataset (fechas, categorías, escalado)
# Eliminar columnas irrelevantes y convertir fecha
df['Fecha_producción'] = pd.to_datetime(df['Fecha_producción'], errors='coerce')
df['Dias_desde_produccion'] = (pd.to_datetime('today') - df['Fecha_producción']).dt.days
df = df.drop(columns=['ID_lote', 'Fecha_producción'])

In [4]:
# Celda 4: Preparar variables predictoras y target
X = df.drop(columns=['Tipo_defecto'])
y = df['Tipo_defecto']

In [5]:
# Celda 5: Codificar variables categóricas
X = pd.get_dummies(X, drop_first=True)

In [6]:
# Celda 6: Imputar valores faltantes (si existen)
imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X)

In [7]:
# Celda 7: Escalar variables numéricas
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

In [8]:
# Celda 8: División en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, stratify=y, random_state=42)

In [9]:
# Celda 9: Definir espacio de búsqueda para Grid Search
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10],
    'min_samples_split': [2, 5],
    'criterion': ['gini', 'entropy']
}

In [10]:
# Celda 10: Aplicar GridSearchCV y medir tiempo
start = time.time()
grid_search = GridSearchCV(RandomForestClassifier(random_state=42,
                                                  class_weight='balanced'),
                           param_grid, scoring='f1_macro', cv=3, n_jobs=-1)
grid_search.fit(X_train, y_train)
end = time.time()
tiempo_grid = end - start
mejor_grid = grid_search.best_estimator_
print('Mejor configuración (Grid Search):', grid_search.best_params_)
print('Tiempo de búsqueda (Grid): {:.2f} segundos'.format(tiempo_grid))

Mejor configuración (Grid Search): {'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 200}
Tiempo de búsqueda (Grid): 419.63 segundos


In [11]:
# Celda 11: Evaluar el modelo obtenido con Grid Search
y_pred_grid = mejor_grid.predict(X_test)
print(classification_report(y_test, y_pred_grid))

                  precision    recall  f1-score   support

   Contaminación       0.15      0.19      0.17      2268
Dosis incorrecta       0.11      0.18      0.13      1516
          Rotura       0.14      0.22      0.17      2264
     Sin defecto       0.59      0.40      0.47      8952

        accuracy                           0.32     15000
       macro avg       0.25      0.25      0.24     15000
    weighted avg       0.41      0.32      0.35     15000



In [12]:
# Celda 12: Definir espacio para Random Search (más amplio y con distribuciones)
from scipy.stats import randint
param_dist = {
    'n_estimators': randint(100, 300),
    'max_depth': randint(4, 15),
    'min_samples_split': randint(2, 10),
    'criterion': ['gini', 'entropy']
}

In [13]:
# Celda 13: Aplicar RandomizedSearchCV y medir tiempo
start = time.time()
random_search = RandomizedSearchCV(RandomForestClassifier(random_state=42,
                                                          class_weight='balanced'),
                                  param_distributions=param_dist,
                                  n_iter=10, scoring='f1_macro', cv=3, n_jobs=-1,
                                  random_state=42)

random_search.fit(X_train, y_train)
end = time.time()
tiempo_random = end - start
mejor_random = random_search.best_estimator_
print('Mejor configuración (Random Search):', random_search.best_params_)
print('Tiempo de búsqueda (Random): {:.2f} segundos'.format(tiempo_random))

Mejor configuración (Random Search): {'criterion': 'gini', 'max_depth': 13, 'min_samples_split': 9, 'n_estimators': 287}
Tiempo de búsqueda (Random): 389.59 segundos


In [14]:
# Celda 14: Evaluar el modelo obtenido con Random Search
y_pred_random = mejor_random.predict(X_test)
print(classification_report(y_test, y_pred_random))

                  precision    recall  f1-score   support

   Contaminación       0.15      0.14      0.15      2268
Dosis incorrecta       0.10      0.11      0.10      1516
          Rotura       0.14      0.16      0.15      2264
     Sin defecto       0.60      0.57      0.58      8952

        accuracy                           0.40     15000
       macro avg       0.25      0.25      0.25     15000
    weighted avg       0.41      0.40      0.40     15000



### **Conclusiones**
- **Grid Search** recorre todas las combinaciones posibles, lo que garantiza encontrar el mejor resultado **dentro del grid definido**, pero puede ser costoso en tiempo.
- **Random Search** prueba combinaciones aleatorias y es más eficiente cuando el espacio de búsqueda es muy grande.
- En este caso, se puede observar si los modelos obtenidos con ambos métodos tienen rendimientos similares o si alguno supera al otro significativamente.

### **Preguntas para reflexionar**
- ¿Qué estrategia usarías si tienes poco tiempo para entrenar?
- ¿Vale la pena recorrer todas las combinaciones posibles?
- ¿Cómo podrías definir un espacio de búsqueda más inteligente para Random Search?